In [1]:
# Working in Google Colab
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [81.0 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,079 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame. (Our source dataset for analysis.)
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)

# Take a look at the DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [14]:
# Use that temporary view to determine
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# Since I'm supposed to practice SQL commands here

spark.sql("SELECT date_built,ROUND(AVG(price),2) FROM home_sales WHERE bedrooms==4 GROUP BY date_built").show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2016|           296050.24|
|      2012|           298233.42|
|      2017|           296576.69|
|      2014|           299073.89|
|      2013|           299999.39|
|      2011|            302141.9|
|      2015|           307908.86|
|      2010|           296800.75|
+----------+--------------------+



In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# That's how the question was worded in the instructions, but I find it a little confusing.
# I'm thinking it means, do the same thing as above but for 3 bed 3 bath homes.

spark.sql("SELECT date_built,ROUND(AVG(price),2) FROM home_sales WHERE bedrooms==3 AND bathrooms==3 GROUP BY date_built").show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2016|           290555.07|
|      2012|           293683.19|
|      2017|           292676.79|
|      2014|           290852.27|
|      2013|           295962.27|
|      2011|           291117.47|
|      2015|            288770.3|
|      2010|           292859.62|
+----------+--------------------+



In [16]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places?
# That's certainly a choice of parameters.

spark.sql("SELECT date_built,ROUND(AVG(price),2) FROM home_sales WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_living>=2000 GROUP BY date_built").show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2016|            293965.1|
|      2012|           307539.97|
|      2017|           280317.58|
|      2014|           298264.72|
|      2013|           303676.79|
|      2011|           276553.81|
|      2015|           297609.97|
|      2010|           285010.22|
+----------+--------------------+



In [18]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# Again, this wording is confusing to me. Is it asking for the average view rating of a home where the price is greater than or equal to $350,000?
# That's how I'll interpret it.

# For measuring runtime:
start_time = time.time()

spark.sql("SELECT ROUND(AVG(price),2),ROUND(AVG(view),2) FROM home_sales WHERE price>=350000").show()

print("Runtime: %s seconds" % (time.time()-start_time))

+--------------------+-------------------+
|round(avg(price), 2)|round(avg(view), 2)|
+--------------------+-------------------+
|           473796.26|              32.26|
+--------------------+-------------------+

Runtime: 0.684617280960083 seconds


In [20]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales")

# 8. Check to see if it worked.

spark.catalog.isCached("home_sales")

True

In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Will this wording ever make sense to me?
# I'm taking this to mean, do the same thing again and compare runtimes.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(price),2),ROUND(AVG(view),2) FROM home_sales WHERE price>=350000").show()

print("Runtime: %s seconds" % (time.time()-start_time))

+--------------------+-------------------+
|round(avg(price), 2)|round(avg(view), 2)|
+--------------------+-------------------+
|           473796.26|              32.26|
+--------------------+-------------------+

Runtime: 0.8545410633087158 seconds


In [22]:
# That did not help me, unfortunately.
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# First I have to make the parquet home sales data, since I've just been going wild till now.

# Write the data in parquet format
df.write.parquet('parquet_homes',mode='overwrite')

# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('parquet_homes')

# Take a look.
parquet_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [23]:
# That sure does look the same.
# 12. Create a temporary table for the parquet data.

parquet_df.createOrReplaceTempView('parquet_home_sales')

In [24]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# For measuring runtime:
start_time = time.time()

spark.sql("SELECT ROUND(AVG(price),2),ROUND(AVG(view),2) FROM parquet_home_sales WHERE price>=350000").show()

print("Runtime: %s seconds" % (time.time()-start_time))

+--------------------+-------------------+
|round(avg(price), 2)|round(avg(view), 2)|
+--------------------+-------------------+
|           473796.26|              32.26|
+--------------------+-------------------+

Runtime: 0.9098680019378662 seconds


In [25]:
# This did nothing to help me, but it was fun to try out.
# 14. Uncache the home_sales temporary table.

spark.sql("uncache table home_sales")

DataFrame[]

In [26]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False